## Combine text


In [ ]:
!wget https://raw.githubusercontent.com/huang-0505/LLM/refs/heads/main/addtional_cleaned_augmented_combined.txt
# read it in to inspect it
with open('addtional_cleaned_augmented_combined.txt', 'r', encoding='utf-8') as f:
    text = f.read()
!pip install gradio

--2025-02-10 21:48:38--  https://raw.githubusercontent.com/huang-0505/LLM/refs/heads/main/addtional_cleaned_augmented_combined.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9563762 (9.1M) [text/plain]
Saving to: ‘addtional_cleaned_augmented_combined.txt’

addtional_cleaned_a 100%[===================>]   9.12M  --.-KB/s    in 0.1s    

2025-02-10 21:48:39 (65.6 MB/s) - ‘addtional_cleaned_augmented_combined.txt’ saved [9563762/9563762]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 89.4 MB/s eta 0:00:00
   ━━

In [ ]:
# Install necessary libraries
!pip install nltk transformers sentence-transformers gradio bertopic spacy -q
!python -m spacy download en_core_web_sm -q

# Import required modules
import os
import re
from transformers import pipeline
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
from bertopic import BERTopic
import pandas as pd

# Curation pipeline
def text_curation_pipeline(text_file, output_dir, min_length=5, max_length=128, clustering_clusters=5, deduplication_threshold=0.8):
    # Step 1: Load Raw Text
    with open(text_file, "r", encoding="utf-8") as file:
        raw_text = file.read()

    # Step 2: Split Text into Chunks
    def split_text_into_chunks(text, chunk_size=500_000):
        return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    chunks = split_text_into_chunks(raw_text)
    sentences = []

    # Step 3: Process Each Chunk with SpaCy
    nlp = spacy.load("en_core_web_sm")
    nlp.max_length = 10_000_000  # Increase max length to handle large texts
    for chunk in chunks:
        doc = nlp(chunk)
        sentences.extend([sent.text for sent in doc.sents])

    # Step 4: Filter Sentences by Length
    filtered_sentences = [s for s in sentences if min_length <= len(s.split()) <= max_length]

    # Step 5: Semantic Deduplication
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(filtered_sentences)
    similarities = cosine_similarity(embeddings)
    unique_sentences = []
    added_indices = set()
    for i, row in enumerate(similarities):
        if i not in added_indices:
            unique_sentences.append(filtered_sentences[i])
            similar_indices = np.where(row > deduplication_threshold)[0]
            added_indices.update(similar_indices)

    # Step 6: Clustering Sentences
    clustering_model = KMeans(n_clusters=clustering_clusters)
    cluster_labels = clustering_model.fit_predict(embeddings[:len(unique_sentences)])

    # Step 7: Extract Advanced Features
    extracted_features = []
    for sentence in unique_sentences:
        doc = nlp(sentence)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        extracted_features.append({"sentence": sentence, "entities": entities})

    # Step 8: Categorization Using BERTopic
    topic_model = BERTopic()
    topics, _ = topic_model.fit_transform(unique_sentences)

    # Save results for training
    os.makedirs(output_dir, exist_ok=True)

    # Save unique sentences for training
    curated_data_path = os.path.join(output_dir, "curated_data3.txt")
    with open(curated_data_path, "w", encoding="utf-8") as f:
        for sentence in unique_sentences:
            f.write(sentence + "\n")
    print(f"Curated data saved to {curated_data_path}")

    # Save embeddings for training
    embeddings_path = os.path.join(output_dir, "embeddings.npy")
    np.save(embeddings_path, embeddings)
    print(f"Embeddings saved to {embeddings_path}")

    # Save metadata (cluster labels and extracted features)
    metadata = {
        "sentence": unique_sentences,
        "cluster_label": cluster_labels,
        "named_entities": extracted_features,
        "topics": topics
    }
    metadata_df = pd.DataFrame(metadata)
    metadata_path = os.path.join(output_dir, "metadata.csv")
    metadata_df.to_csv(metadata_path, index=False)
    print(f"Metadata saved to {metadata_path}")

    return curated_data_path, embeddings_path, metadata_path

# Run the pipeline on your text file and save results for training
curated_data_file, embeddings_file, metadata_file = text_curation_pipeline(
    "synthetic_data.txt",
    output_dir="./curated_data",
    min_length=5,
    max_length=128,
    clustering_clusters=5,
    deduplication_threshold=0.8
)

print(f"Pipeline completed! Outputs:\n- Curated Data: {curated_data_file}\n- Embeddings: {embeddings_file}\n- Metadata: {metadata_file}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

# Replace with a smaller model
MODEL_NAME = "EleutherAI/gpt-neo-1.3B"
DATA_FILE = "curated_data.txt"
OUTPUT_FILE = "synthetic_data2.txt"
NUM_PROMPTS = 2  # Reduced number of synthetic examples per prompt
BATCH_SIZE = 16  # Smaller batch size for faster processing
TEMPERATURE = 0.7  # Sampling diversity
TOP_P = 0.9  # Nucleus sampling
MAX_LENGTH = 256  # Reduced sequence length for faster generation

# Load model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Ensure pad_token is set
tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token

# Load model with mixed precision or smaller memory allocation
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None  # Offload to manage memory
)

# Load curated data
print("Loading curated data...")
with open(DATA_FILE, "r", encoding="utf-8") as file:
    curated_data = [line.strip() for line in file.readlines() if line.strip()]

# Reduce data size for faster processing
curated_data = curated_data[20001:30000]  # Only use the first 100 lines for data generation

# Generate synthetic data
print("Generating synthetic data...")
synthetic_data = []

for batch_start in tqdm(range(0, len(curated_data), BATCH_SIZE), desc="Processing batches"):
    batch = curated_data[batch_start:batch_start + BATCH_SIZE]

    # Tokenize batch
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)

    # Generate synthetic examples for the batch
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=MAX_LENGTH,
        num_return_sequences=NUM_PROMPTS,
        do_sample=True,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        pad_token_id=tokenizer.pad_token_id
    )

    # Decode and collect synthetic examples
    for i, output in enumerate(outputs):
        generated_text = tokenizer.decode(output, skip_special_tokens=True)
        synthetic_data.append(generated_text)

# Save synthetic data
print(f"Saving synthetic data to {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w", encoding="utf-8") as file:
    for example in synthetic_data:
        file.write(example + "\n")

print(f"Synthetic data generation complete. Saved {len(synthetic_data)} examples to {OUTPUT_FILE}.")


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

Loading curated data...
Generating synthetic data...


Processing batches: 100%|██████████| 625/625 [53:54<00:00,  5.18s/it]

Saving synthetic data to synthetic_data2.txt...
Synthetic data generation complete. Saved 19998 examples to synthetic_data2.txt.


In [ ]:
CURATED_FILE = "combined_curated_data.txt"
SYNTHETIC_FILE = "synthetic_data2.txt"
COMBINED_FILE = "combined_curated_data3.txt"

# Combine the files
with open(CURATED_FILE, "r", encoding="utf-8") as curated, \
     open(SYNTHETIC_FILE, "r", encoding="utf-8") as synthetic, \
     open(COMBINED_FILE, "w", encoding="utf-8") as combined:

    # Write curated data
    combined.write("Curated Data:\n")
    combined.write(curated.read())
    combined.write("\n\n")

    # Write synthetic data
    combined.write("Synthetic Data:\n")
    combined.write(synthetic.read())

print(f"Combined data saved to {COMBINED_FILE}.")


Combined data saved to combined_curated_data3.txt.


In [ ]:
# Define chunk size (approximate word count per chunk)
chunk_size = 50

# Read the file
file_path = 'addtional_cleaned_augmented_combined.txt'  # Replace with your file path if necessary
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Split content into smaller chunks based on approximate word count
words = content.split()
chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# Prepare the chunks with metadata
def extract_keywords(text, num_keywords=5):
    from sklearn.feature_extraction.text import TfidfVectorizer

    # TF-IDF vectorizer for extracting keywords
    vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_array = vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.toarray().flatten()
    top_indices = tfidf_scores.argsort()[-num_keywords:][::-1]
    keywords = [feature_array[idx] for idx in top_indices]
    return ', '.join(keywords)

knowledge_base = []
for idx, chunk in enumerate(chunks):
    keywords = extract_keywords(chunk)  # Extract keywords for the chunk
    knowledge_base.append({
        "chunk_id": f"chunk_{idx + 1:04}",
        "source": "Cleaned_Augmented_Combined",
        "start_position": idx * chunk_size,  # Start position in the word list
        "content": chunk,
        "keywords": keywords  # Add extracted keywords as metadata
    })

# Convert to a DataFrame for easier exploration
import pandas as pd
knowledge_base_df = pd.DataFrame(knowledge_base)

# Display the DataFrame
knowledge_base_df.head()  # Use .head() to preview the first few rows

# Save the DataFrame to a CSV for later use (optional)
knowledge_base_df.to_csv('enhanced_knowledge_base.csv', index=False)

print("Enhanced knowledge base created with metadata and analysis! You can now explore or export it.")


Enhanced knowledge base created with metadata and analysis! You can now explore or export it.


In [ ]:
# from tokenizers import ByteLevelBPETokenizer
# import os

# # Step 1: Create and train the tokenizer
# tokenizer = ByteLevelBPETokenizer()
# tokenizer.train(
#     files=["addtional_cleaned_augmented_combined.txt"],
#     vocab_size=30000,
#     special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
# )

# # Step 2: Ensure the directory exists and save the tokenizer
# os.makedirs("./my_tokenizer", exist_ok=True)
# tokenizer.save_model("./my_tokenizer")

# print("Tokenizer successfully saved!")


### Combined Text

In [ ]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F
from tokenizers import ByteLevelBPETokenizer, Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import PreTrainedTokenizerFast

# hyperparameters
batch_size = 16
block_size = 128
max_iters = 5000
eval_interval = 100
learning_rate = 5e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.3

torch.manual_seed(1337)

# Step 1: Train a ByteLevelBPETokenizer
print("Training ByteLevelBPETokenizer...")
tokenizer = ByteLevelBPETokenizer()
save_dir = "./my_custom_tokenizer"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

tokenizer.train(
    files=["curated_data.txt"],  # Use the curated data file here
    vocab_size=10000,
    special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
)

tokenizer.save_model(save_dir)
print("Tokenizer saved as vocab.json and merges.txt")

# Convert vocab.json and merges.txt to tokenizer.json
print("Converting vocab.json and merges.txt to tokenizer.json format...")
vocab_path = f"{save_dir}/vocab.json"
merges_path = f"{save_dir}/merges.txt"

tokenizer = Tokenizer(models.BPE.from_file(vocab_path, merges_path))
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.post_processor = processors.TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> $B:1 </s>:1",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ],
)

tokenizer.save(f"{save_dir}/tokenizer.json")
print("Tokenizer converted to tokenizer.json format!")

# Step 2: Load the tokenizer using PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file=f"{save_dir}/tokenizer.json",
    model_max_length=block_size,
    pad_token="<pad>"
)

tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.unk_token = "<unk>"
tokenizer.pad_token = "<pad>"
tokenizer.mask_token = "<mask>"

print("Special tokens:")
print(f"BOS token: {tokenizer.bos_token} -> {tokenizer.bos_token_id}")
print(f"EOS token: {tokenizer.eos_token} -> {tokenizer.eos_token_id}")
print(f"PAD token: {tokenizer.pad_token} -> {tokenizer.pad_token_id}")
print(f"UNK token: {tokenizer.unk_token} -> {tokenizer.unk_token_id}")

vocab_size = tokenizer.vocab_size
print("New vocabulary size:", vocab_size)

# Step 3: Load and process the curated dataset
with open('curated_data.txt', 'r', encoding='utf-8') as f:  # Use the curated data file
    text = f.read()

# Tokenize the curated dataset and pad chunks
print("Tokenizing curated data...")
data = tokenizer.encode(text)

chunks = []
for i in range(0, len(data), block_size):
    chunk = data[i:i + block_size]
    if len(chunk) < block_size:  # Pad shorter chunks
        chunk += [tokenizer.pad_token_id] * (block_size - len(chunk))
    chunks.append(chunk)

data = torch.tensor(chunks, dtype=torch.long)
print("Tokenized curated data shape:", data.shape)

# Train and Test Splits
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    d = train_data if split == 'train' else val_data
    ix = torch.randint(len(d) - 1, (batch_size,))
    x = torch.stack([d[i] for i in ix]).to(device)
    y = torch.stack([d[i + 1] for i in ix]).to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, n_layer, block_size, dropout):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Initialize the BigramLanguageModel with required arguments
model = BigramLanguageModel(
    vocab_size=vocab_size,
    n_embd=n_embd,
    n_head=n_head,
    n_layer=n_layer,
    block_size=block_size,
    dropout=dropout
).to(device)

print(sum(p.numel() for p in model.parameters()) / 1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate Text
context = torch.tensor([[tokenizer.bos_token_id]], dtype=torch.long, device=device)
print(tokenizer.decode(model.generate(context, max_new_tokens=200)[0].tolist()))


## Multihead Latent Attention

In [ ]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F
from tokenizers import ByteLevelBPETokenizer, Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import PreTrainedTokenizerFast

# hyperparameters
batch_size = 8
block_size = 512
max_iters = 2000
eval_interval = 100
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 512
n_head = 4
n_layer = 4
latent_dim = 64  # Dimension for MLA latent representations
dropout = 0.3

torch.manual_seed(1337)

# Step 1: Train a ByteLevelBPETokenizer
print("Training ByteLevelBPETokenizer...")
tokenizer = ByteLevelBPETokenizer()
save_dir = "./my_custom_tokenizer"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

tokenizer.train(
    files=["combined_curated_data3.txt"],  # Use the curated data file here
    vocab_size=10000,
    special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
)

tokenizer.save_model(save_dir)
print("Tokenizer saved as vocab.json and merges.txt")

# Convert vocab.json and merges.txt to tokenizer.json
print("Converting vocab.json and merges.txt to tokenizer.json format...")
vocab_path = f"{save_dir}/vocab.json"
merges_path = f"{save_dir}/merges.txt"

tokenizer = Tokenizer(models.BPE.from_file(vocab_path, merges_path))
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.post_processor = processors.TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> $B:1 </s>:1",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ],
)

tokenizer.save(f"{save_dir}/tokenizer.json")
print("Tokenizer converted to tokenizer.json format!")

# Step 2: Load the tokenizer using PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file=f"{save_dir}/tokenizer.json",
    model_max_length=block_size,
    pad_token="<pad>"
)

tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.unk_token = "<unk>"
tokenizer.pad_token = "<pad>"
tokenizer.mask_token = "<mask>"

vocab_size = tokenizer.vocab_size

# Step 3: Load and process the curated dataset
with open('combined_curated_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Tokenize the curated dataset and pad chunks
data = tokenizer.encode(text)

chunks = []
for i in range(0, len(data), block_size):
    chunk = data[i:i + block_size]
    if len(chunk) < block_size:
        chunk += [tokenizer.pad_token_id] * (block_size - len(chunk))
    chunks.append(chunk)

data = torch.tensor(chunks, dtype=torch.long)
print("Tokenized curated data shape:", data.shape)

# Train and Test Splits
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    d = train_data if split == 'train' else val_data
    ix = torch.randint(len(d) - 1, (batch_size,))
    x = torch.stack([d[i] for i in ix]).to(device)
    y = torch.stack([d[i + 1] for i in ix]).to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Multi-head Latent Attention (MLA)
class MultiHeadLatentAttention(nn.Module):
    def __init__(self, n_embd, num_heads, latent_dim, dropout):
        super().__init__()
        self.num_heads = num_heads
        self.latent_dim = latent_dim
        self.key = nn.Linear(n_embd, num_heads * latent_dim, bias=False)
        self.query = nn.Linear(n_embd, num_heads * latent_dim, bias=False)
        self.value = nn.Linear(n_embd, num_heads * latent_dim, bias=False)
        self.proj = nn.Linear(num_heads * latent_dim, n_embd)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x).view(B, T, self.num_heads, self.latent_dim)
        q = self.query(x).view(B, T, self.num_heads, self.latent_dim)
        v = self.value(x).view(B, T, self.num_heads, self.latent_dim)

        # Transpose dimensions for attention computation
        k = k.transpose(1, 2)  # (B, num_heads, T, latent_dim)
        q = q.transpose(1, 2)  # (B, num_heads, T, latent_dim)
        v = v.transpose(1, 2)  # (B, num_heads, T, latent_dim)

        # Compute scaled dot-product attention
        attn_weights = (q @ k.transpose(-2, -1)) / (self.latent_dim ** 0.5)  # (B, num_heads, T, T)

        # Apply causal mask
        mask = self.tril[:T, :T].unsqueeze(0).unsqueeze(0)  # (1, 1, T, T)
        attn_weights = attn_weights.masked_fill(mask == 0, float('-inf'))

        # Softmax over the last dimension
        attn_weights = F.softmax(attn_weights, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Weighted sum of values
        out = attn_weights @ v  # (B, num_heads, T, latent_dim)
        out = out.transpose(1, 2).contiguous().view(B, T, -1)  # (B, T, num_heads * latent_dim)
        out = self.proj(out)  # (B, T, n_embd)
        return out

# Define Transformer Block with MLA
class Block(nn.Module):
    def __init__(self, n_embd, n_head, latent_dim, dropout):
        super().__init__()
        self.mla = MultiHeadLatentAttention(n_embd, n_head, latent_dim, dropout)
        self.ffwd = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.mla(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# Define the Language Model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, latent_dim, n_layer, block_size, dropout):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, latent_dim, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Initialize and Train the Model
model = BigramLanguageModel(
    vocab_size=vocab_size,
    n_embd=n_embd,
    n_head=n_head,
    latent_dim=latent_dim,
    n_layer=n_layer,
    block_size=block_size,
    dropout=dropout
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate Text
context = torch.tensor([[tokenizer.bos_token_id]], dtype=torch.long, device=device)
print(tokenizer.decode(model.generate(context, max_new_tokens=200)[0].tolist()))


Training ByteLevelBPETokenizer...
Tokenizer saved as vocab.json and merges.txt
Converting vocab.json and merges.txt to tokenizer.json format...
Tokenizer converted to tokenizer.json format!


Token indices sequence length is longer than the specified maximum sequence length for this model (6152847 > 512). Running this sequence through the model will result in indexing errors


Tokenized curated data shape: torch.Size([12018, 512])
step 0: train loss 9.4092, val loss 9.4068
step 100: train loss 6.3187, val loss 5.9991
step 200: train loss 6.2853, val loss 5.9935
step 300: train loss 6.2822, val loss 5.9882
step 400: train loss 6.2818, val loss 5.9618
step 500: train loss 6.2642, val loss 6.0007
step 600: train loss 6.2537, val loss 5.9904
step 700: train loss 6.2648, val loss 6.0013
step 800: train loss 6.2424, val loss 6.0352
step 900: train loss 6.2143, val loss 6.0110
step 1000: train loss 6.2313, val loss 6.0173
step 1100: train loss 6.2033, val loss 5.9893
step 1200: train loss 6.1711, val loss 6.0011
step 1300: train loss 6.1879, val loss 5.9962
step 1400: train loss 6.1736, val loss 6.0226
step 1500: train loss 6.1599, val loss 5.9889
step 1600: train loss 6.1549, val loss 6.0413
step 1700: train loss 6.1750, val loss 6.0267
step 1800: train loss 6.1515, val loss 6.0495
step 1900: train loss 6.1540, val loss 6.0277
step 1999: train loss 6.1511, val los

## LLM chain

In [ ]:
# import torch
# import torch.nn as nn
# from torch.nn import functional as F

# # hyperparameters
# batch_size = 32
# block_size = 16
# max_iters = 500
# eval_interval = 100
# learning_rate = 5e-4
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# eval_iters = 200
# n_embd = 256
# n_head = 8
# n_layer = 8
# dropout = 0.1

# torch.manual_seed(1337)

# # Load dataset
# with open('addtional_cleaned_augmented_combined.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

# # Vocabulary and encoding
# chars = sorted(list(set(text)))
# vocab_size = len(chars)
# stoi = {ch: i for i, ch in enumerate(chars)}
# itos = {i: ch for i, ch in enumerate(chars)}
# encode = lambda s: [stoi[c] for c in s]
# decode = lambda l: ''.join([itos[i] for i in l])

# # Train and test splits
# data = torch.tensor(encode(text), dtype=torch.long)
# n = int(0.9 * len(data))
# train_data = data[:n]
# val_data = data[n:]

# # Data loading
# def get_batch(split):
#     data = train_data if split == 'train' else val_data
#     ix = torch.randint(len(data) - block_size, (batch_size,))
#     x = torch.stack([data[i:i + block_size] for i in ix])
#     y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
#     x, y = x.to(device), y.to(device)
#     return x, y

# @torch.no_grad()
# def estimate_loss():
#     out = {}
#     model.eval()
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in range(eval_iters):
#             X, Y = get_batch(split)
#             logits, loss = model(X, Y)
#             losses[k] = loss.item()
#         out[split] = losses.mean()
#     model.train()
#     return out

# # Define Transformer model components
# class Head(nn.Module):
#     def __init__(self, head_size):
#         super().__init__()
#         self.key = nn.Linear(n_embd, head_size, bias=False)
#         self.query = nn.Linear(n_embd, head_size, bias=False)
#         self.value = nn.Linear(n_embd, head_size, bias=False)
#         self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         B, T, C = x.shape
#         k = self.key(x)
#         q = self.query(x)
#         wei = q @ k.transpose(-2, -1) * C**-0.5
#         wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
#         wei = F.softmax(wei, dim=-1)
#         wei = self.dropout(wei)
#         v = self.value(x)
#         out = wei @ v
#         return out

# class MultiHeadAttention(nn.Module):
#     def __init__(self, num_heads, head_size):
#         super().__init__()
#         self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
#         self.proj = nn.Linear(n_embd, n_embd)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         out = torch.cat([h(x) for h in self.heads], dim=-1)
#         out = self.dropout(self.proj(out))
#         return out

# class FeedFoward(nn.Module):
#     def __init__(self, n_embd):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(n_embd, 4 * n_embd),
#             nn.ReLU(),
#             nn.Linear(4 * n_embd, n_embd),
#             nn.Dropout(dropout),
#         )

#     def forward(self, x):
#         return self.net(x)

# class Block(nn.Module):
#     def __init__(self, n_embd, n_head):
#         super().__init__()
#         head_size = n_embd // n_head
#         self.sa = MultiHeadAttention(n_head, head_size)
#         self.ffwd = FeedFoward(n_embd)
#         self.ln1 = nn.LayerNorm(n_embd)
#         self.ln2 = nn.LayerNorm(n_embd)

#     def forward(self, x):
#         x = x + self.sa(self.ln1(x))
#         x = x + self.ffwd(self.ln2(x))
#         return x

# class BigramLanguageModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
#         self.position_embedding_table = nn.Embedding(block_size, n_embd)
#         self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
#         self.ln_f = nn.LayerNorm(n_embd)
#         self.lm_head = nn.Linear(n_embd, vocab_size)

#     def forward(self, idx, targets=None):
#         B, T = idx.shape
#         tok_emb = self.token_embedding_table(idx)
#         pos_emb = self.position_embedding_table(torch.arange(T, device=device))
#         x = tok_emb + pos_emb
#         x = self.blocks(x)
#         x = self.ln_f(x)
#         logits = self.lm_head(x)

#         if targets is None:
#             loss = None
#         else:
#             B, T, C = logits.shape
#             logits = logits.view(B * T, C)
#             targets = targets.view(B * T)
#             loss = F.cross_entropy(logits, targets)

#         return logits, loss

#     def generate(self, idx, max_new_tokens):
#         for _ in range(max_new_tokens):
#             idx_cond = idx[:, -block_size:]
#             logits, loss = self(idx_cond)
#             logits = logits[:, -1, :]
#             probs = F.softmax(logits, dim=-1)
#             idx_next = torch.multinomial(probs, num_samples=1)
#             idx = torch.cat((idx, idx_next), dim=1)
#         return idx

# # Train the model
# model = BigramLanguageModel().to(device)
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# for iter in range(max_iters):
#     if iter % eval_interval == 0 or iter == max_iters - 1:
#         losses = estimate_loss()
#         print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

#     xb, yb = get_batch('train')
#     logits, loss = model(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

# # Define the LLM Chain
# def llm_chain(user_input, model, stoi, itos, block_size, max_new_tokens=100):
#     context = [stoi.get(ch, 0) for ch in user_input]
#     context = torch.tensor(context, dtype=torch.long, device=device).unsqueeze(0)
#     generated = model.generate(context, max_new_tokens=max_new_tokens)
#     output = ''.join([itos[idx] for idx in generated[0].tolist()])
#     return output

# # Example usage of the LLM chain
# user_input = "The quick brown fox"
# print("Generated Output:")
# print(llm_chain(user_input, model, stoi, itos, block_size, max_new_tokens=200))


step 0: train loss 4.7421, val loss 4.7192
step 100: train loss 2.3363, val loss 2.4939
step 200: train loss 2.1498, val loss 2.3535
step 300: train loss 2.0157, val loss 2.2153
step 400: train loss 1.9204, val loss 2.1464
step 499: train loss 1.8605, val loss 2.0767
Generated Output:
The quick brown fox yave will that d bled, anfor chus of ye with willte ye in unto shall. do thim who all vall wave wal wilat. the d wich whow, alp podsest do upoth the hhey and ment in is tho wepherth nechit for ay wki


## save model and use knowledge

In [ ]:
torch.save(model.state_dict(), 'bigram_language_model.pth')
print("Model saved successfully!")

Model saved successfully!


In [ ]:
import torch
from transformers import PreTrainedTokenizerFast
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import gradio as gr

# Load the knowledge base
knowledge_base_path = "enhanced_knowledge_base.csv"  # Path to the CSV file
knowledge_base = pd.read_csv(knowledge_base_path)

# Load embeddings for the knowledge base
retriever_model = SentenceTransformer("all-MiniLM-L6-v2")
knowledge_base_embeddings = retriever_model.encode(knowledge_base["content"].tolist(), convert_to_tensor=True)

# Load the tokenizer
save_dir = "./my_custom_tokenizer"
tokenizer = PreTrainedTokenizerFast(tokenizer_file=f"{save_dir}/tokenizer.json")
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = "<pad>"

# Load the trained model
vocab_size = tokenizer.vocab_size
n_embd = 512
n_head = 4
n_layer = 4
block_size = 512
dropout = 0.3

class MultiHeadLatentAttention(torch.nn.Module):
    def __init__(self, n_embd, num_heads, latent_dim, dropout):
        super().__init__()
        self.num_heads = num_heads
        self.latent_dim = latent_dim
        self.key = torch.nn.Linear(n_embd, num_heads * latent_dim, bias=False)
        self.query = torch.nn.Linear(n_embd, num_heads * latent_dim, bias=False)
        self.value = torch.nn.Linear(n_embd, num_heads * latent_dim, bias=False)
        self.proj = torch.nn.Linear(num_heads * latent_dim, n_embd)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x).view(B, T, self.num_heads, self.latent_dim).transpose(1, 2)
        q = self.query(x).view(B, T, self.num_heads, self.latent_dim).transpose(1, 2)
        v = self.value(x).view(B, T, self.num_heads, self.latent_dim).transpose(1, 2)

        attn_weights = (q @ k.transpose(-2, -1)) / (self.latent_dim ** 0.5)
        attn_weights = attn_weights.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        attn_weights = torch.nn.functional.softmax(attn_weights, dim=-1)
        attn_weights = self.dropout(attn_weights)

        out = attn_weights @ v
        out = out.transpose(1, 2).contiguous().view(B, T, -1)
        out = self.proj(out)
        return out

class Block(torch.nn.Module):
    def __init__(self, n_embd, n_head, latent_dim, dropout):
        super().__init__()
        self.mla = MultiHeadLatentAttention(n_embd, n_head, latent_dim, dropout)
        self.ffwd = torch.nn.Sequential(
            torch.nn.Linear(n_embd, 4 * n_embd),
            torch.nn.ReLU(),
            torch.nn.Linear(4 * n_embd, n_embd),
            torch.nn.Dropout(dropout),
        )
        self.ln1 = torch.nn.LayerNorm(n_embd)
        self.ln2 = torch.nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.mla(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(torch.nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, latent_dim, n_layer, block_size, dropout):
        super().__init__()
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = torch.nn.Embedding(block_size, n_embd)
        self.blocks = torch.nn.Sequential(*[Block(n_embd, n_head, latent_dim, dropout) for _ in range(n_layer)])
        self.ln_f = torch.nn.LayerNorm(n_embd)
        self.lm_head = torch.nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = torch.nn.functional.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = torch.nn.functional.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel(
    vocab_size=vocab_size,
    n_embd=n_embd,
    n_head=n_head,
    latent_dim=64,
    n_layer=n_layer,
    block_size=block_size,
    dropout=dropout,
).to("cuda" if torch.cuda.is_available() else "cpu")

model.load_state_dict(torch.load("bigram_language_model.pth"))
model.eval()

# Retrieval function
def retrieve_information(prompt, top_k=3):
    try:
        prompt_embedding = retriever_model.encode(prompt, convert_to_tensor=True)
        similarities = util.cos_sim(prompt_embedding.cpu(), knowledge_base_embeddings.cpu())
        top_k_indices = torch.topk(similarities, k=top_k).indices
        return [knowledge_base["content"].iloc[idx] for idx in top_k_indices[0]]
    except Exception as e:
        return [f"Error during retrieval: {e}"]

# Gradio interface
def generate_text_with_knowledge(prompt, max_new_tokens):
    try:
        # Retrieve relevant information
        retrieved_chunks = retrieve_information(prompt)
        print("Retrieved chunks:", retrieved_chunks)  # Debug statement

        # Augment the prompt with retrieved information
        augmented_prompt = prompt + "\n\n" + "\n".join(retrieved_chunks)
        print("Augmented prompt:", augmented_prompt)  # Debug statement

        # Generate text using the augmented prompt
        device = "cuda" if torch.cuda.is_available() else "cpu"
        input_ids = tokenizer(augmented_prompt, return_tensors="pt").input_ids.to(device)
        generated_ids = model.generate(input_ids, max_new_tokens=max_new_tokens)
        return tokenizer.decode(generated_ids[0].tolist(), skip_special_tokens=True)
    except Exception as e:
        return f"Error: {e}"

iface = gr.Interface(
    fn=generate_text_with_knowledge,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your prompt here..."),
        gr.Slider(10, 200, step=10, label="Max New Tokens"),
    ],
    outputs="text",
    title="Text Generation with Knowledge Base",
    description="Enter a prompt, and the model will retrieve relevant knowledge to generate text.",
)

iface.launch()

<ipython-input-16-a4fc528fe453>:123: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("bigram_language_model.pth"))


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cf7246231f8816264b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# import pandas as pd
# from sentence_transformers import SentenceTransformer, util
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # --------- Step 1: Load Knowledge Base from CSV ---------
# # Load the CSV file
# knowledge_base_df = pd.read_csv('enhanced_knowledge_base.csv')  # Replace with the actual path to your CSV file

# # Extract the "content" column as a list of documents
# knowledge_base = knowledge_base_df['content'].tolist()

# # --------- Step 2: Build Knowledge Base Embeddings ---------
# # Initialize the SentenceTransformer model
# embedder = SentenceTransformer('all-MiniLM-L6-v2')

# # Create embeddings for the knowledge base
# knowledge_embeddings = embedder.encode(knowledge_base, convert_to_tensor=True)

# # --------- Step 3: Define Retrieval Function ---------
# def retrieve_documents(query, top_k=3):
#     # Encode the query
#     query_embedding = embedder.encode(query, convert_to_tensor=True)

#     # Compute cosine similarity
#     scores = util.cos_sim(query_embedding, knowledge_embeddings)[0]
#     top_results = torch.topk(scores, k=top_k)

#     # Retrieve top-k documents
#     retrieved_docs = [knowledge_base[i] for i in top_results.indices]
#     return retrieved_docs

# # --------- Step 4: Define Question-Answer Function ---------
# def ask_question(question, model, tokenizer, top_k=3, max_new_tokens=50, device='cpu'):
#     """
#     Asks a question using the RAG model by retrieving relevant documents
#     and generating an answer based on the context.
#     """
#     # Retrieve relevant documents from the knowledge base
#     retrieved_docs = retrieve_documents(question, top_k=top_k)

#     # Combine retrieved documents into a context
#     context = "<retrieval> " + " ".join(retrieved_docs) + " <query> " + question

#     # Encode the context
#     context_text = f"<bos> Q: {context}\nA: "
#     input_ids = tokenizer.encode(context_text, return_tensors='pt').to(device)

#     # Generate the response
#     generated_outputs = model.generate(
#         input_ids=input_ids,
#         max_length=input_ids.shape[1] + max_new_tokens,
#         do_sample=True,
#         temperature=0.7,
#         num_return_sequences=1,
#         eos_token_id=tokenizer.eos_token_id
#     )

#     # Decode the generated tokens into text
#     generated_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)

#     # Extract the answer (after "A:")
#     answer = generated_text.split("A:")[1].strip() if "A:" in generated_text else generated_text.strip()
#     return answer

# # --------- Step 5: Interactive Question Loop ---------
# def interactive_loop(model, tokenizer, device='cpu'):
#     """
#     Interactive loop for asking questions to the bot.
#     """
#     print("Interactive Question-Answering Bot is ready! (Type 'exit' to quit)\n")
#     while True:
#         user_question = input("Ask a question: ")
#         if user_question.lower() == "exit":
#             print("Goodbye!")
#             break
#         try:
#             answer = ask_question(user_question, model, tokenizer, device=device)
#             print(f"A: {answer}\n")
#         except Exception as e:
#             print(f"Error: {e}\n")

# # # --------- Step 6: Run the Interactive Bot ---------
# # # Initialize your language model and tokenizer
# # device = 'cuda' if torch.cuda.is_available() else 'cpu'

# # # Replace 'gpt2' with the model of your choice
# # tokenizer = AutoTokenizer.from_pretrained('gpt2')
# # model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)

# # # Start the interactive loop
# # interactive_loop(model, tokenizer, device=device)


In [ ]:
# # Initialize your language model and tokenizer
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# # Replace 'gpt2' with the model of your choice
# tokenizer = AutoTokenizer.from_pretrained('gpt2')
# model = AutoModelForCausalLM.from_pretrained('gpt2').to(device)

# # Start the interactive loop
# interactive_loop(model, tokenizer, device=device)


In [ ]:
!pip install gradio

In [ ]:
# import pandas as pd
# import torch
# import torch.nn as nn
# from torch.nn import functional as F
# from transformers import AutoTokenizer
# from sentence_transformers import SentenceTransformer, util
# import gradio as gr

# # --------- Step 1: Preprocess the Dataset ---------
# # Define chunk size (approximate word count per chunk)
# chunk_size = 50

# # Read the file
# file_path = 'addtional_cleaned_augmented_combined.txt'  # Replace with your file path if necessary
# with open(file_path, 'r', encoding='utf-8') as file:
#     content = file.read()

# # Split content into smaller chunks based on approximate word count
# words = content.split()
# chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

# # Prepare the chunks with metadata
# def extract_keywords(text, num_keywords=5):
#     from sklearn.feature_extraction.text import TfidfVectorizer

#     # TF-IDF vectorizer for extracting keywords
#     vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
#     tfidf_matrix = vectorizer.fit_transform([text])
#     feature_array = vectorizer.get_feature_names_out()
#     tfidf_scores = tfidf_matrix.toarray().flatten()
#     top_indices = tfidf_scores.argsort()[-num_keywords:][::-1]
#     keywords = [feature_array[idx] for idx in top_indices]
#     return ', '.join(keywords)

# knowledge_base = []
# for idx, chunk in enumerate(chunks):
#     keywords = extract_keywords(chunk)  # Extract keywords for the chunk
#     knowledge_base.append({
#         "chunk_id": f"chunk_{idx + 1:04}",
#         "source": "Cleaned_Augmented_Combined",
#         "start_position": idx * chunk_size,  # Start position in the word list
#         "content": chunk,
#         "keywords": keywords  # Add extracted keywords as metadata
#     })

# # Convert to a DataFrame for easier exploration
# knowledge_base_df = pd.DataFrame(knowledge_base)

# # Save the DataFrame to a CSV for later use
# knowledge_base_df.to_csv('enhanced_knowledge_base.csv', index=False)
# print("Enhanced knowledge base created with metadata and analysis!")

# # --------- Step 2: Initialize the Knowledge Base Embeddings ---------
# # Load knowledge base
# knowledge_base = knowledge_base_df['content'].tolist()

# # Initialize the SentenceTransformer model
# embedder = SentenceTransformer('all-MiniLM-L6-v2')

# # Create embeddings for the knowledge base
# knowledge_embeddings = embedder.encode(knowledge_base, convert_to_tensor=True)

# # --------- Step 3: Retrieval-based Augmentation ---------
# def retrieve_documents(query, top_k=3):
#     query_embedding = embedder.encode(query, convert_to_tensor=True)
#     scores = util.cos_sim(query_embedding, knowledge_embeddings)[0]
#     top_results = torch.topk(scores, k=top_k)
#     retrieved_docs = [knowledge_base[i] for i in top_results.indices]
#     return retrieved_docs

# # --------- Step 4: Define the BigramLanguageModel ---------
# class Head(nn.Module):
#     def __init__(self, head_size):
#         super().__init__()
#         self.key = nn.Linear(n_embd, head_size, bias=False)
#         self.query = nn.Linear(n_embd, head_size, bias=False)
#         self.value = nn.Linear(n_embd, head_size, bias=False)
#         self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         B, T, C = x.shape
#         k = self.key(x)
#         q = self.query(x)
#         wei = q @ k.transpose(-2, -1) * C ** -0.5
#         wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
#         wei = F.softmax(wei, dim=-1)
#         wei = self.dropout(wei)
#         v = self.value(x)
#         return wei @ v

# class MultiHeadAttention(nn.Module):
#     def __init__(self, num_heads, head_size):
#         super().__init__()
#         self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
#         self.proj = nn.Linear(n_embd, n_embd)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x):
#         return self.dropout(self.proj(torch.cat([h(x) for h in self.heads], dim=-1)))

# class FeedForward(nn.Module):
#     def __init__(self, n_embd):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(n_embd, 4 * n_embd),
#             nn.ReLU(),
#             nn.Linear(4 * n_embd, n_embd),
#             nn.Dropout(dropout),
#         )

#     def forward(self, x):
#         return self.net(x)

# class Block(nn.Module):
#     def __init__(self, n_embd, n_head):
#         super().__init__()
#         head_size = n_embd // n_head
#         self.sa = MultiHeadAttention(n_head, head_size)
#         self.ffwd = FeedForward(n_embd)
#         self.ln1 = nn.LayerNorm(n_embd)
#         self.ln2 = nn.LayerNorm(n_embd)

#     def forward(self, x):
#         return x + self.ffwd(self.ln2(x + self.sa(self.ln1(x))))

# class BigramLanguageModel(nn.Module):
#     def __init__(self, vocab_size, n_embd, n_head, n_layer, block_size, dropout):
#         super().__init__()
#         self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
#         self.position_embedding_table = nn.Embedding(block_size, n_embd)
#         self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
#         self.ln_f = nn.LayerNorm(n_embd)
#         self.lm_head = nn.Linear(n_embd, vocab_size)

#     def forward(self, idx, targets=None):
#         B, T = idx.shape
#         tok_emb = self.token_embedding_table(idx)
#         pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
#         x = self.blocks(tok_emb + pos_emb)
#         logits = self.lm_head(self.ln_f(x))
#         loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1)) if targets is not None else None
#         return logits, loss

# # --------- Step 5: Train or Load the Model ---------
# # Training hyperparameters
# n_embd, n_head, n_layer, block_size, dropout = 64, 2, 2, 16, 0.5
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# tokenizer = AutoTokenizer.from_pretrained("gpt2")

# # Initialize and load model
# model = BigramLanguageModel(
#     vocab_size=tokenizer.vocab_size,
#     n_embd=n_embd,
#     n_head=n_head,
#     n_layer=n_layer,
#     block_size=block_size,
#     dropout=dropout
# ).to(device)

# try:
#     model.load_state_dict(torch.load("bigram_language_model.pth"))
#     model.eval()
#     print("Model loaded successfully!")
# except FileNotFoundError:
#     print("No pre-trained model found. Please train your model first!")

# # --------- Step 6: Launch Gradio Chatbot ---------
# def chat_with_model(user_input):
#     # Retrieve relevant documents
#     retrieved_docs = retrieve_documents(user_input, top_k=3)
#     context = " ".join(retrieved_docs) + " " + user_input
#     input_ids = tokenizer(context, return_tensors="pt", truncation=True)["input_ids"].to(device)
#     output_ids = model.generate(input_ids[:, -block_size:], max_new_tokens=100)
#     return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# with gr.Blocks() as chatbot_ui:
#     gr.Markdown("# Knowledge-Augmented Chatbot")
#     user_input = gr.Textbox(label="Enter your message:")
#     bot_response = gr.Textbox(label="Chatbot's response:")
#     submit_button = gr.Button("Send")
#     submit_button.click(chat_with_model, inputs=user_input, outputs=bot_response)

# chatbot_ui.launch()


### Gardio Interface

In [ ]:
import torch

# Save the model state_dict
torch.save(model.state_dict(), 'bigram_language_model.pth')

# Save the tokenizer
tokenizer.save_pretrained('./my_tokenizer')

# Save the model configuration
config = {
    'vocab_size': vocab_size,
    'n_embd': n_embd,
    'n_head': n_head,
    'n_layer': n_layer,
    'block_size': block_size,
    'dropout': dropout
}
torch.save(config, 'model_config.pth')

print("Model, tokenizer, and configuration saved successfully!")

import torch
import gradio as gr
from transformers import PreTrainedTokenizerFast
from torch.nn import functional as F

# Define the BigramLanguageModel class (ensure this matches your training code)
class BigramLanguageModel(torch.nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, n_layer, block_size, dropout):
        super().__init__()
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = torch.nn.Embedding(block_size, n_embd)
        self.blocks = torch.nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = torch.nn.LayerNorm(n_embd)
        self.lm_head = torch.nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            logits = logits.view(B * T, -1)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

class Block(torch.nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        self.sa = MultiHeadAttention(n_head, n_embd // n_head)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = torch.nn.LayerNorm(n_embd)
        self.ln2 = torch.nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class MultiHeadAttention(torch.nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = torch.nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = torch.nn.Linear(num_heads * head_size, num_heads * head_size)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class Head(torch.nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = torch.nn.Linear(64, head_size, bias=False)
        self.query = torch.nn.Linear(64, head_size, bias=False)
        self.value = torch.nn.Linear(64, head_size, bias=False)
        self.dropout = torch.nn.Dropout(0.5)
        self.register_buffer("tril", torch.tril(torch.ones(128, 128)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = (q @ k.transpose(-2, -1)) * (1.0 / (k.size(-1) ** 0.5))
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        return wei @ v

class FeedForward(torch.nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(n_embd, 4 * n_embd),
            torch.nn.ReLU(),
            torch.nn.Linear(4 * n_embd, n_embd),
            torch.nn.Dropout(0.5),
        )

    def forward(self, x):
        return self.net(x)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained('./my_tokenizer')

# Load model configuration
config = torch.load('model_config.pth')

# Initialize the model
model = BigramLanguageModel(**config).to(device)

# Load the model weights
state_dict = torch.load('bigram_language_model.pth', map_location=device)
model.load_state_dict(state_dict, strict=True)
model.eval()

print("Model and tokenizer loaded successfully!")

# Define the Gradio interface
def gradio_chat(user_input):
    # Tokenize the user input
    input_ids = tokenizer(user_input, return_tensors="pt").input_ids.to(device)

    # Generate a response
    output_ids = model.generate(input_ids, max_new_tokens=50)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return response

# Launch Gradio app
interface = gr.Interface(
    fn=gradio_chat,
    inputs=gr.Textbox(label="Enter your input"),
    outputs=gr.Textbox(label="Model's response"),
    title="Bigram Language Model Chatbot"
)

interface.launch()


Model, tokenizer, and configuration saved successfully!
Using device: cuda


<ipython-input-9-9ffff9f8bd23>:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  config = torch.load('model_config.pth')
<ipython-input-9-9ffff9f8bd23>:135: FutureWarning: 

RuntimeError: Error(s) in loading state_dict for BigramLanguageModel:
	Missing key(s) in state_dict: "blocks.0.sa.heads.0.tril", "blocks.0.sa.heads.0.key.weight", "blocks.0.sa.heads.0.query.weight", "blocks.0.sa.heads.0.value.weight", "blocks.0.sa.heads.1.tril", "blocks.0.sa.heads.1.key.weight", "blocks.0.sa.heads.1.query.weight", "blocks.0.sa.heads.1.value.weight", "blocks.0.sa.heads.2.tril", "blocks.0.sa.heads.2.key.weight", "blocks.0.sa.heads.2.query.weight", "blocks.0.sa.heads.2.value.weight", "blocks.0.sa.heads.3.tril", "blocks.0.sa.heads.3.key.weight", "blocks.0.sa.heads.3.query.weight", "blocks.0.sa.heads.3.value.weight", "blocks.0.sa.proj.weight", "blocks.0.sa.proj.bias", "blocks.0.ffwd.net.0.weight", "blocks.0.ffwd.net.0.bias", "blocks.0.ffwd.net.2.weight", "blocks.0.ffwd.net.2.bias", "blocks.1.sa.heads.0.tril", "blocks.1.sa.heads.0.key.weight", "blocks.1.sa.heads.0.query.weight", "blocks.1.sa.heads.0.value.weight", "blocks.1.sa.heads.1.tril", "blocks.1.sa.heads.1.key.weight",

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
from torch.nn import functional as F
from transformers import AutoTokenizer
import gradio as gr

# --------- Load Knowledge Base ---------
knowledge_base_df = pd.read_csv('enhanced_knowledge_base.csv')
knowledge_base = knowledge_base_df['content'].tolist()

# --------- Build Knowledge Base Embeddings ---------
embedder = SentenceTransformer('all-MiniLM-L6-v2')
knowledge_embeddings = embedder.encode(knowledge_base, convert_to_tensor=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]